# Анализ базы данных сервиса для чтения книг

<font size="4"><b>О проекте:</b></font>

В настоящее время у людей появился повышенный интерес к книгам, в связи с чем актуальным стало создание удобных приложений для чтения книг. Наша компания приобрела сервис для чтения книг по подписке с целью создания нового продукта. Первый этап достижения этой цели - исследование приобретенной базы данных, это задача данного исследования.

<font size="4"><b>Описание данных:</b></font>

База данных образована 5-ю таблицами.

1. Таблица `books` - содержит данные о книгах:

    * `book_id` — идентификатор книги;
    * `author_id` — идентификатор автора;
    * `title` — название книги;
    * `num_pages` — количество страниц;
    * `publication_date` — дата публикации книги;
    * `publisher_id` — идентификатор издателя.
    
    
2. Таблица `authors` - содержит данные об авторах:
    * `author_id` — идентификатор автора;
    * `author` — имя автора.
    
   
3. Таблица `publishers` - содержит данные об издательствах:
    * `publisher_id` — идентификатор издательства;
    * `publisher` — название издательства.
    
    
4. Таблица `ratings` - содержит данные о пользовательских оценках книг: 
    * `rating_id` — идентификатор оценки;
    * `book_id` — идентификатор книги;
    * `username` — имя пользователя, оставившего оценку;
    * `rating` — оценка книги.
    
    
5. Таблица `reviews` - содержит данные о пользовательских обзорах: 
    * `review_id` — идентификатор обзора;
    * `book_id` — идентификатор книги;
    * `username` — имя автора обзора;
    * `text` — текст обзора.

<font size="4"><b>Схема данных:</b></font>

<img src=https://pictures.s3.yandex.net/resources/scheme_1589269096.png>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Общая-информация" data-toc-modified-id="Общая-информация-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Общая информация</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Определение-функций" data-toc-modified-id="Определение-функций-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Определение функций</a></span><ul class="toc-item"><li><span><a href="#df_info()" data-toc-modified-id="df_info()-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span><code>df_info()</code></a></span></li><li><span><a href="#query_this()" data-toc-modified-id="query_this()-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span><code>query_this()</code></a></span></li></ul></li><li><span><a href="#Общая-информация-о-данных" data-toc-modified-id="Общая-информация-о-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Общая информация о данных</a></span></li></ul></li><li><span><a href="#Исследование-БД" data-toc-modified-id="Исследование-БД-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование БД</a></span><ul class="toc-item"><li><span><a href="#Количество-книг,-выпущенных-в-21-м-веке" data-toc-modified-id="Количество-книг,-выпущенных-в-21-м-веке-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Количество книг, выпущенных в 21-м веке</a></span></li><li><span><a href="#Количество-обзоров-и-средняя-оценка-каждой-книги" data-toc-modified-id="Количество-обзоров-и-средняя-оценка-каждой-книги-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Количество обзоров и средняя оценка каждой книги</a></span></li><li><span><a href="#Самое-масштабное-издательство" data-toc-modified-id="Самое-масштабное-издательство-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Самое масштабное издательство</a></span></li><li><span><a href="#Самый-успешный-автор" data-toc-modified-id="Самый-успешный-автор-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Самый успешный автор</a></span></li><li><span><a href="#Количество-обзоров-самых-активных-пользователей" data-toc-modified-id="Количество-обзоров-самых-активных-пользователей-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Количество обзоров самых активных пользователей</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Общая информация

### Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# подключаемся к БД

# устанавливаем параметры
# (параметры скрыты)
db_config = {
    'user': ,    # имя пользователя
    'pwd': ,        # пароль
    'host': ,
    'port': ,                   # порт подключения
    'db':      # название базы данных
} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
con = engine.connect()

### Определение функций

#### `df_info()`

In [3]:
# создадим функцию для первичного ознакомления с данными 
def df_info(table):
    
    """Функция выводит первые 5 строк таблицы, а также количество записей в таблице"""
    
    # формируем запросы
    query_5_str = f'''SELECT * FROM {table} LIMIT 5'''
    query_count = f'''SELECT COUNT(*) FROM {table}'''
    
    # обращаемся к БД
    df = pd.io.sql.read_sql(sql=text(query_5_str), con=con)
    df_size = pd.io.sql.read_sql(sql=text(query_count), con=con)
    
    # выводим информацию
    print(f'Количество записей в таблице {table}:', df_size.loc[0, 'count'])
    display(df)
    print()
    
    # посмотрим также на типы данных в столбцах
    print('Типы данных столбцов:')
    query_type = f'''
    SELECT 
        table_name, 
        column_name, 
        data_type
    FROM information_schema.columns 
    WHERE table_name = '{table}';
    '''
    display(pd.io.sql.read_sql(query_type, con =con))
    print()
    
    # проверим наличие пустых значений
    print('Пустые значения в столбцах:')
    columns = (pd.io.sql
               .read_sql(
                   f'''SELECT column_name FROM information_schema.columns WHERE table_name = '{table}';''',
                   con=con
               )['column_name']
               .to_list()
              )
    for column in columns:
        
        query_null = f'''
        SELECT 
            COUNT(*)
        FROM {table}
        WHERE {column} IS NULL;
        '''
        count_of_nulls = pd.io.sql.read_sql(query_null, con =con)
        
        # выведем количество пустых значений в столбце
        print(f'Количество пустых значений в столбце {column}: {count_of_nulls.loc[0, "count"]}')

#### `query_this()`

In [4]:
# создадим функцию для удобной отправки запроса к БД
def query_this(query):
    """
    Функция для подключения к базе данных с запросом, переданным этой функции. Функция возвращает датафрейм.
    """
    data = pd.io.sql.read_sql(sql=text(query), con=con)
    return data

### Общая информация о данных

Посмотрим на общий вид данных.

In [5]:
df_info('books')

Количество записей в таблице books: 1000


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Типы данных столбцов:


,table_name,column_name,data_type
0,books,book_id,integer
1,books,author_id,integer
2,books,num_pages,integer
3,books,publication_date,date
4,books,publisher_id,integer
5,books,title,text



Пустые значения в столбцах:
Количество пустых значений в столбце book_id: 0
Количество пустых значений в столбце author_id: 0
Количество пустых значений в столбце num_pages: 0
Количество пустых значений в столбце publication_date: 0
Количество пустых значений в столбце publisher_id: 0
Количество пустых значений в столбце title: 0


В БД есть информация о 1000 книг. Все типы столбцов соответствуют своим данным, пропусков нет.

In [6]:
df_info('authors')

Количество записей в таблице authors: 636


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Типы данных столбцов:


,table_name,column_name,data_type
0,authors,author_id,integer
1,authors,author,text



Пустые значения в столбцах:
Количество пустых значений в столбце author_id: 0
Количество пустых значений в столбце author: 0


В БД присутствует информация о книгах 636 авторов, типы соответствуют своим данным, пропусков нет.

In [7]:
df_info('publishers')

Количество записей в таблице publishers: 340


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Типы данных столбцов:


,table_name,column_name,data_type
0,publishers,publisher_id,integer
1,publishers,publisher,text



Пустые значения в столбцах:
Количество пустых значений в столбце publisher_id: 0
Количество пустых значений в столбце publisher: 0


В БД информация о 340 изданиях, пустых значений нет, типы соответствуют своим данным.

In [8]:
df_info('ratings')

Количество записей в таблице ratings: 6456


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Типы данных столбцов:


,table_name,column_name,data_type
0,ratings,rating_id,integer
1,ratings,book_id,integer
2,ratings,rating,integer
3,ratings,username,text



Пустые значения в столбцах:
Количество пустых значений в столбце rating_id: 0
Количество пустых значений в столбце book_id: 0
Количество пустых значений в столбце rating: 0
Количество пустых значений в столбце username: 0


В БД присутствует информация о ~6,5 тыс. оценок, типы соответствуют своим данным, пропусков нет.

In [9]:
df_info('reviews')

Количество записей в таблице reviews: 2793


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



Типы данных столбцов:


,table_name,column_name,data_type
0,reviews,review_id,integer
1,reviews,book_id,integer
2,reviews,username,text
3,reviews,text,text



Пустые значения в столбцах:
Количество пустых значений в столбце review_id: 0
Количество пустых значений в столбце book_id: 0
Количество пустых значений в столбце username: 0
Количество пустых значений в столбце text: 0


В БД присутствует информация о ~2,8 тыс. обзорах, типы соответствуют своим данным, пропусков нет.

## Исследование БД

### Количество книг, выпущенных в 21-м веке

In [10]:
print('Количество книг, выпущенных после 1 января 2000 года составляет',
      query_this(
        '''
        SELECT COUNT(*)
        FROM books
        WHERE publication_date > '2000-01-01'
        '''
        ).loc[0, 'count'],
      'штук(у/и).'
    )

Количество книг, выпущенных после 1 января 2000 года составляет 819 штук(у/и).


### Количество обзоров и средняя оценка каждой книги

In [11]:
# для того, чтобы учесть возможное отсутствие обзоров или оценок, при формировании ошибок укажем,
# чтобы пустые значения заполнились нулями

books_ratings_and_reviews = query_this(
    '''
    WITH
        book_ratings AS (
            SELECT 
                book_id,
                AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY book_id
        ),
        book_revs AS (
            SELECT
                book_id,
                COUNT(review_id) AS count_of_revs
            FROM reviews
            GROUP BY book_id
        )
        
    SELECT
        title, 
        CASE
            WHEN avg_rating IS NOT NULL THEN avg_rating
            WHEN avg_rating IS NULL THEN 0
        END AS avg_rating,
        CASE
            WHEN count_of_revs IS NOT NULL THEN count_of_revs
            WHEN count_of_revs IS NULL THEN 0
        END AS count_of_revs
    FROM books AS b
    LEFT JOIN book_ratings AS brat ON b.book_id = brat.book_id
    LEFT JOIN book_revs AS br ON b.book_id = br.book_id
    ORDER BY avg_rating DESC, count_of_revs DESC
    '''
)

In [12]:
books_ratings_and_reviews.head(10)

,title,avg_rating,count_of_revs
0,A Dirty Job (Grim Reaper #1),5.0,4
1,Moneyball: The Art of Winning an Unfair Game,5.0,3
2,School's Out—Forever (Maximum Ride #2),5.0,3
3,The Woman in Black,5.0,2
4,Arrows of the Queen (Heralds of Valdemar #1),5.0,2
5,Geek Love,5.0,2
6,Stolen (Women of the Otherworld #2),5.0,2
7,Plum Island (John Corey #1),5.0,2
8,Captivating: Unveiling the Mystery of a Woman'...,5.0,2
9,The Ghost Map: The Story of London's Most Terr...,5.0,2


Таблица с количеством рейтингов и отзывов успешно составлена. Можно отметить, что самой популярной книгой оказалась первая часть Сумерек.

### Самое масштабное издательство

In [13]:
query_this(
    '''
    WITH
        counts_of_books AS (
            SELECT 
                publisher_id,
                COUNT(book_id) AS count_of_books
            FROM books
            WHERE num_pages > 50
            GROUP BY publisher_id
        )
        
    SELECT publisher, count_of_books
    FROM publishers AS p
        JOIN counts_of_books AS c ON p.publisher_id = c.publisher_id
    ORDER BY count_of_books DESC
    LIMIT 1
    '''
)

,publisher,count_of_books
0,Penguin Books,42


Больше всего книг было выпущено Penguin Books.

### Самый успешный автор

In [14]:
query_this(
    '''
    WITH
        avg_rating_of_books AS (
            SELECT 
                book_id,
                AVG(rating) AS avg_rating,
                COUNT(rating)
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(rating) >= 50
        ),
        
        best_authors AS (
            SELECT 
                author_id,
                AVG(avg_rating) AS avg_rating_of_books
            FROM books AS b
                RIGHT JOIN avg_rating_of_books AS avg ON b.book_id = avg.book_id
            WHERE num_pages >= 50
            GROUP BY author_id
        )
            
        SELECT
            author,
            avg_rating_of_books
        FROM best_authors AS ba
            JOIN authors AS a ON ba.author_id = a.author_id
        ORDER BY avg_rating_of_books DESC
        LIMIT 1
    '''
)

,author,avg_rating_of_books
0,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка книг у Роулинг - автора серии книг о Гарри Поттере.

### Количество обзоров самых активных пользователей

In [15]:
print('Самые активные пользователи в среднем составляют',
    
      query_this(
    '''
    WITH
        rev_counts AS (
            SELECT COUNT(review_id) AS counts_of_rev
            FROM reviews
            WHERE username IN (
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48
                )
            GROUP BY username
    )
    
    SELECT CAST(ROUND(AVG(counts_of_rev)) AS int)
    FROM rev_counts
    '''
    ).loc[0, 'round'],
      
      'обзора.'
)

Самые активные пользователи в среднем составляют 24 обзора.


## Вывод

В таблицах нет пропусков, все данные хранятся в корректном виде.

На все поставленные вопросы был получен ответ:
1. после 1 янв 2000 года было выпущено 819 книг;
2. в проекте создана таблица с информацией о количестве обзоров и средней оценке каждой книги данной БД;
3. больше всего книг (42 шт.) было выпущено изданием Penguin Books;
4. самая высокая средняя оценка книг у J.K. Rowling (4.3);
5. самые активные пользователи в среднем составляют 24 обзора.